#### Data Loading

In [34]:
import os
import numpy as np
import pandas as pd
import pickle
import lightgbm as lgb
#from lightgbm import LGBMRegressor  # sckit에서 lightgbm을 wrapping한 것
from sklearn.model_selection import train_test_split
import datetime
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import Utility as Util

########
buildingID, buildingName = 'B0001', '용학초등학교'
file_path = '/home/ymatics/CodingSpace/2024_AI_BEMS/241014_data_colec_h/data_colec_h_202410141046_B0006.csv'
#########
df_buildID = pd.read_csv(file_path)
df_buildID['colec_dt'] = pd.to_datetime(df_buildID['colec_dt']).dt.floor('min')  # 분 이하는 제거
print(f"timestamp(first~last): {df_buildID['colec_dt'].iloc[0]} ~ {df_buildID['colec_dt'].iloc[-1]}")
print(f"timestamp(min~max):    {df_buildID['colec_dt'].min()} ~ {df_buildID['colec_dt'].max()}")

start_date, end_date = pd.to_datetime('2023-01-01 00:00:00'), pd.to_datetime('2024-10-14 10:00:00')
df_raw_all = df_buildID[(df_buildID['colec_dt'] >= start_date) & (df_buildID['colec_dt'] <= end_date)]

devID, device_name = 2001, '학교MAIN'

Util.print_tagCD(df_raw_all, devID)
tag_dict = {30001: '현재 출력'}
tag_dict = {40004: '데이터 유효성 체크'}  # 이 것으로 missing_ratio 관찰
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df_raw_all, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]
Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=True)


import B0008_MainPower_DataProcessing_241111 as DP

df_raw = pd.DataFrame(data=tag_data[0]['colec_val'].values, index=tag_data[0]['colec_dt'], columns=['value'])
#df_raw = df_raw.iloc[int(len(df_raw)/2-1500):-1].copy()
print(f"timestamp: {df_raw.index[0]}, {df_raw.index[-1]}")
#df_interpol = Util.resample_time_index_interpolate_NaN_df(df_raw, '15min', 'linear')
#Util.plot_interpolated_data(df_raw, df_interpol, device_name, createFig=True)
#Util.plot_data(df_raw, plotType='simple', title=None, W=10, H=5)

X_df, y_df, nan_counts_df, missing_ratio = DP.preprocess(df_raw, points=4, freqInterval='15min')
print(f"nan_counts max= {nan_counts_df.max()}, {missing_ratio= }")

print(f"missing_ratio= {missing_ratio*100:.1f}")

timestamp(first~last): 2023-01-01 00:00:00 ~ 2024-10-14 10:30:00
timestamp(min~max):    2023-01-01 00:00:00 ~ 2024-10-14 10:30:00
tagCD list:  [202202  30001  40004  50001]


timestamp: 2023-01-01 00:00:00, 2024-05-02 15:00:00
nan_counts max= 677, missing_ratio= 0.000
missing_ratio= 0.0
